In [1]:
# avoid warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
#imports
import pandas as pd
from google.cloud import bigquery

In [3]:
#params
GCP_PROJECT = os.environ.get('GCP_PROJECT')
BQ_DATASET = os.environ.get('BQ_DATASET')
BUCKET_NAME = os.environ.get('BUCKET_NAME')
GOOGLE_APPLICATION_CREDENTIALS = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')

In [4]:
# get table name
full_table_name = f'{GCP_PROJECT}.{BQ_DATASET}.image_data_balanced'
# write query
query = f'''
        SELECT 
            artwork_id,
            title,
            category,
            medium,
            date,
            height_cm,
            width_cm,
            image_url_template,
            collecting_institution,
            image_url_normalized
        FROM {full_table_name}
        '''

In [5]:
# get image table from big query
# instantiate client
client = bigquery.Client(project=GCP_PROJECT)
# set up query job
query_job = client.query(query)
# run query
result = query_job.result()
# store results in df
artworks_df = result.to_dataframe()

In [10]:
artworks_df.head(1)

,artwork_id,title,category,medium,date,height_cm,width_cm,image_url_template,collecting_institution,image_url_normalized
0,4eb1c899c8004a000100deb3,Portrait of a Young Woman with a White Coif,Painting,Oil and tempera on panel,1541,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/liVgLUFu...,None,https://d32dm0rphc51dk.cloudfront.net/liVgLUFu...


In [13]:
artworks_df.to_csv('bq_data.csv',index=False)

In [9]:
#create function to return relevant image data in a library
sim_indexes = [0, 4, 67, 9943, 6002]

#artworks_df.loc[2100,:]
artworks_df[artworks_df['category'] == 'Painting']
#,'category','medium','date','collecting_institution'

,artwork_id,title,category,medium,date,height_cm,width_cm,image_url_template,collecting_institution,image_url_normalized
0,4eb1c899c8004a000100deb3,Portrait of a Young Woman with a White Coif,Painting,Oil and tempera on panel,1541,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/liVgLUFu...,None,https://d32dm0rphc51dk.cloudfront.net/liVgLUFu...
1,4f99da873314020001000700,The Alba Madonna,Painting,Oil on panel transferred to canvas,ca. 1510,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/lnnzsg3v...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/lnnzsg3v...
2,515b1d46056351dc33001014,The Fall of Man [middle panel],Painting,Oil on hardboard transferred from panel,ca. 1535,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/kBOdyvyl...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/kBOdyvyl...
3,515ce23e7b7057eb4c00115c,The Rule of Mars [right panel],Painting,Oil on hardboard transferred from panel,ca. 1535,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/hidKPTZj...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/hidKPTZj...
4,515d6a15b5907b33b1004797,Madonna and Child with Saint Mary Magdalene an...,Painting,Tempera on panel transferred to canvas,ca. 1330/1340,NaN,NaN,https://d32dm0rphc51dk.cloudfront.net/-SeTFEk1...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/-SeTFEk1...
...,...,...,...,...,...,...,...,...,...,...
5491,515b47769562c8fee200247d,Judith with the Head of Holofernes,Painting,Tempera on panel,ca. 1495/1500,30.8,19.7,https://d32dm0rphc51dk.cloudfront.net/FTRtd-FH...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/FTRtd-FH...
5492,515ce5635eeb1c904c0011ca,Saint Matthew,Painting,Tempera on panel,probably c. 1320,30.8,23.2,https://d32dm0rphc51dk.cloudfront.net/jlnZBe3m...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/jlnZBe3m...
5493,515d24c57b7057eb4c0032fd,Madonna and Child,Painting,Tempera on panel,ca. 1470,67.2,46.0,https://d32dm0rphc51dk.cloudfront.net/Tj5syD80...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/Tj5syD80...
5494,515cd5427696593fde000a8b,Madonna and Child with Saint Anthony Abbot and...,Painting,Tempera on panel,ca. 1490/1495,158.4,141.9,https://d32dm0rphc51dk.cloudfront.net/R9knXRK7...,"National Gallery of Art, Washington D.C.",https://d32dm0rphc51dk.cloudfront.net/R9knXRK7...


In [7]:
for index, row in artworks_df.iterrows():
    if 'star' in row['title']:
        print(index) 


In [23]:
artworks_df.iloc[4150,:]

artwork_id                                         506c958a3445b400020011eb
title                                        Doctor Paul Gachet (1828-1909)
category                                                           Painting
medium                                                        Oil on canvas
date                                                                   1890
height_cm                                                              68.0
width_cm                                                               57.0
image_url_template        https://d32dm0rphc51dk.cloudfront.net/_5K0xSYg...
collecting_institution                                 Musée d'Orsay, Paris
image_url_normalized      https://d32dm0rphc51dk.cloudfront.net/_5K0xSYg...
Name: 4150, dtype: object